In [1]:
import pandas as pd
import pathlib
import os
from tqdm import tqdm

PROJECT = "gs-mrd"

maindir = "/media/hieunguyen/GSHD_HN01"
path_to_storage = os.path.join(maindir, "storage")
path_to_main_src = "/media/hieunguyen/HNSD01/src/gs-mrd"
path_to_save_merge_feature = os.path.join(path_to_main_src, "all_samples")
path_to_model_files = os.path.join(path_to_main_src, "model_files")

os.system(f"mkdir -p {path_to_save_merge_feature}")


0

# Read the metadata, convert labcode


In [2]:
##### modify metadata
if os.path.isfile("All Samples GW_MRD_010924.modified.xlsx") == False:
    metadata = pd.read_excel("All Samples GW_MRD_010924.xlsx")
    '''
    These labcodes have been changed and need to be converted in the metadata.
    '''
    convert_labcode = {
        "HMAAAA03": "ZTKL01A",
        "HMAAAA26": "ZTKL05A",
        "HMAAAA21": "ZTKL07A",
        "ZMC031A": "ZMC031",
        "ZMC057A": "ZMC057",
        "ZMC005A": "ZMC005",
        "ZMG093A": "ZMG093",
        "MDCAAA03": "MQCAAA03",
        "MDAAAA18": "MQAAAA18",
        "ZMG040A": "ZMC040A"
    }
    metadata["SampleID"] = metadata["SampleID"].apply(lambda x: convert_labcode[x] if x in convert_labcode.keys() else x)
    metadata = metadata[~metadata["SampleID"].duplicated()]
    metadata.to_excel("All Samples GW_MRD_010924.modified.xlsx")
else:
    metadata = pd.read_excel("All Samples GW_MRD_010924.modified.xlsx")
    
metadata = metadata[~metadata["SampleID"].duplicated()]

print(f"Number of samples in metadata: {metadata.shape[0]}")

Number of samples in metadata: 1550


# Prepare metadata for the Release version 20240606

In [3]:
merge_data_version = "20240914"
release_version = "06062024"

path_to_batch_metadata = os.path.join(path_to_save_merge_feature, merge_data_version)
path_to_release_data = os.path.join(path_to_model_files, release_version)

os.system(f"mkdir -p {path_to_release_data}")

all_metadata_files = [item for item in pathlib.Path(path_to_batch_metadata).rglob("*_batch_metadata.csv")]

# Note: use feature EM as reference metadata
metadata = pd.read_csv([item for item in all_metadata_files if "EM" in item.name][0])
metadata1 = metadata[metadata["Group_RUN"] == release_version]

# remove rerun sampels in this batch
rerun_samples = pd.read_csv("rerun_samples_not_used.txt", header = None)[0].unique()
metadata1 = metadata1[metadata1["SampleID"].isin(rerun_samples) == False]
# metadata1["SampleID"] = metadata1["SampleID"].apply(lambda x: x.split("-")[1])

metadata1.to_csv(os.path.join(path_to_release_data, "release_metadata.csv"), index = False)
metadata1.shape

(388, 3)

In [7]:
metadata

,SampleID,RUN,Group_RUN
0,7-ZMC057,Research-mrdgw-all-batch2,06062024
1,8-ZMG093,Research-mrdgw-all-batch2,06062024
2,6-ZMC071A,Research-mrdgw-all-batch2,06062024
3,6-MDGAAA16,Research-mrdgw-all-batch2,06062024
4,8-ZMC046B,Research-mrdgw-all-batch2,06062024
...,...,...,...
1720,54-ZK0AAAH44NB,batch_180824,from_ECD_WGS
1721,1-ZK0AAAI47NB,batch_180824,from_ECD_WGS
1722,83-ZK0AAAH57NB,batch_180824,from_ECD_WGS
1723,84-ZK0AAAH58NB,batch_180824,from_ECD_WGS
